In [24]:
from nltk.translate.bleu_score import sentence_bleu
import pandas
import pickle
from tqdm import tqdm
import numpy as np

In [35]:
!pip install -U sentence-transformers

  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached huggingface_hub-0.21.4-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.15.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.4.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached threadpoolctl-3.3.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
Using cached huggingface_hub-0.21.4-py3-none-any.whl (346 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached scikit_learn-1.3.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 7.5 MB/s eta 0:00:00
Using cached safetensors-0.4.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014

In [36]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
def calculate_metrics(result_df):
  output_bleu_scores = []
  cosine_similarities = []
  for _, row in tqdm(result_df.iterrows()):
    model_response = row['model_response']
    expected_response = row['expected_response']
    output_bleu_scores.append(sentence_bleu([expected_response.split()], model_response.split()))
    
    model_encoding = model.encode(model_response)
    expected_encoding = model.encode(expected_response)
    consine_similarity = np.dot(model_encoding, expected_encoding)/(np.linalg.norm(model_encoding)*np.linalg.norm(expected_encoding))
    cosine_similarities.append(consine_similarity)
  output_bleu_scores = np.array(output_bleu_scores)
  cosine_similarities = np.array(cosine_similarities)
  return np.mean(output_bleu_scores), np.mean(cosine_similarities)

In [43]:
# result_paths = ['lora_finetune_r_8', 'lora_finetune_r_16', 'rosa_finetune_r_16', 'rosa_finetune_r_8']
result_paths = [ 'rosa_finetune_r_8']
BLEU_score = []
cosine_score = []
for result_path in result_paths:
  print("For experiment:", result_path)
  with open(f"./{result_path}.pkl", "rb") as input_file:
    result_df = pickle.load(input_file)
    bleu, cosine = calculate_metrics(result_df)
    BLEU_score.append(bleu)
    cosine_score.append(cosine)

For experiment: lora_finetune_r_8_2


0it [00:00, ?it/s]

/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower orde

For experiment: lora_finetune_r_8


0it [00:00, ?it/s]/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

...

0it [00:00, ?it/s]/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

For experiment: rosa_finetune_r_16


0it [00:00, ?it/s]/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

For experiment: lora_finetune_r_8


0it [00:00, ?it/s]/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/macvincent/miniconda3/envs/local_nmt/lib/python3.8/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

In [44]:
print(result_paths)
print(BLEU_score)
print(cosine_score)

['lora_finetune_r_8_2', 'lora_finetune_r_8', 'lora_finetune_r_16', 'rosa_finetune_r_16', 'lora_finetune_r_8']
[0.006843055269802055, 0.006843055269802055, 0.006843055269802055, 0.005146332659836869, 0.006843055269802055]
[0.43039066, 0.4303719, 0.42997456, 0.41129872, 0.4303719]
